## Find transformation between GT and GT2 for Patient 27

In [1]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform
import os
# os.makedirs("numpyTests")

# Load NIfTI images (segmentations with 4 classes: background, heart, etc.)
image_1_nii = nib.load('data/segthor_train/train/Patient_27/GT.nii.gz')  # Source image
image_2_nii = nib.load('data/segthor_train/train/Patient_27/GT2.nii.gz')  # Transformed image

# Convert to NumPy arrays
image_1 = image_1_nii.get_fdata()
image_2 = image_2_nii.get_fdata()

# Heart label = 2
heart_mask_1 = (image_1 == 2)  
heart_mask_2 = (image_2 == 2)  

# Get the coordinates of the heart region in both images
coords_1 = np.array(np.nonzero(heart_mask_1)).T
coords_2 = np.array(np.nonzero(heart_mask_2)).T

# Compute the center of mass of the heart region in both images
center_1 = coords_1.mean(axis=0)
center_2 = coords_2.mean(axis=0)

# Translation vector for the heart
translation_vector = center_2 - center_1

# Create an affine transformation matrix
affine_matrix = np.eye(4)  # Identity matrix for rotation/scaling
affine_matrix[:3, 3] = translation_vector  # Set translation part

## Find rotation between GT and GT2 for Patient 27

In [2]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

def apply_small_rotation(mask, axis='z', angle_degrees=27, center_of_mass=None):
    """
    Apply a small rotation around the specified axis.
    """
    # Convert the angle to radians
    angle_radians = np.deg2rad(angle_degrees)
    
    # Define rotation matrices for each axis
    if axis == 'x':
        rotation_matrix = np.array([[1, 0, 0],
                                    [0, np.cos(angle_radians), -np.sin(angle_radians)],
                                    [0, np.sin(angle_radians), np.cos(angle_radians)]])
    elif axis == 'y':
        rotation_matrix = np.array([[np.cos(angle_radians), 0, np.sin(angle_radians)],
                                    [0, 1, 0],
                                    [-np.sin(angle_radians), 0, np.cos(angle_radians)]])
    else:  # Default to 'z' axis
        rotation_matrix = np.array([[np.cos(angle_radians), -np.sin(angle_radians), 0],
                                    [np.sin(angle_radians), np.cos(angle_radians), 0],
                                    [0, 0, 1]])
    
    # Step 1: Translate the center of mass to the origin
    translation_to_origin = np.eye(4)
    translation_to_origin[:3, 3] = -center_of_mass

    # Step 2: Apply the rotation
    full_affine = np.eye(4)
    full_affine[:3, :3] = rotation_matrix
    full_affine = np.dot(full_affine, translation_to_origin)

    # Step 3: Translate back to the center of mass
    translation_back = np.eye(4)
    translation_back[:3, 3] = center_of_mass
    full_affine = np.dot(translation_back, full_affine)

    # Step 4: Apply the small rotation
    transformed_mask = affine_transform(mask.astype(float),
                                        full_affine[:3, :3], 
                                        offset=full_affine[:3, 3], 
                                        order=0,  # Nearest neighbor to preserve mask labels
                                        mode='nearest')
    return transformed_mask

## Apply rotation + translation for all other patients

In [3]:
import os

# Base directory for patient data
base_dir = 'data/segthor_train/train/'

# Loop over all patient folders (Patient_01 to Patient_40)
for patient_id in range(1, 41):
    patient_folder = os.path.join(base_dir, f'Patient_{patient_id:02d}')

    # Load the new image where you want to apply the transformation
    image_path = os.path.join(patient_folder, 'GT.nii.gz')
    new_image_nii = nib.load(image_path)
    new_image = new_image_nii.get_fdata()

    ## ROTATION
    # Heart label = 2 (as in previous images)
    heart_mask_new = (new_image == 2)  # Isolate the heart region in the new image
    coords_new = np.array(np.nonzero(heart_mask_new)).T  # Get coordinates of the heart region

    # Compute the center of mass of the heart region
    center_new = coords_new.mean(axis=0)

    # Apply a small rotation around the z-axis (or any other axis)
    transformed_heart = apply_small_rotation(heart_mask_new, axis='z', angle_degrees=27, center_of_mass=center_new)

    # Binarize the transformed heart region to ensure it's binary
    transformed_heart = transformed_heart > 0.5  # Thresholding to create a binary mask

    # Remove the original heart region from the new image
    new_image_no_heart = np.where(heart_mask_new, 0, new_image)  # Set the heart region to zero

    # Reassign the label (2) to the transformed heart region and combine with the image
    new_image = np.where(transformed_heart, 2, new_image_no_heart)

    ### ------------------------

    ## TRANSLATION

    # Heart label = 2 (as in previous images)
    heart_mask_new = (new_image == 2)  # Isolate the heart region in the new image
    heart_region_new = new_image * heart_mask_new  # Extract heart region

    # Apply translation/affine transformation to the heart region
    # To maintain labels, apply transformation directly to mask and reassign label
    transformed_heart = affine_transform(heart_mask_new.astype(float), 
                                        affine_matrix[:3, :3], 
                                        offset=-translation_vector, 
                                        order=1)

    # Binarize the transformed heart region to avoid interpolation issues
    transformed_heart = transformed_heart > 0.5  # Thresholding to create a binary mask

    # Remove the heart region from the original new image to avoid overlap
    new_image_no_heart = np.where(heart_mask_new, 0, new_image)  # Set the heart region to zero

    # Reassign the label (2) to the transformed heart and add it back into the image
    final_image = np.where(transformed_heart, 2, new_image_no_heart)

    # Save the transformed image in the patient's folder
    output_path = os.path.join(patient_folder, 'GT_corrected.nii.gz')
    final_img_nii = nib.Nifti1Image(final_image, new_image_nii.affine)
    nib.save(final_img_nii, output_path)

## Check IOU between given and transformed corrected GT for Patient 27

In [4]:
import nibabel as nib
import numpy as np

def compute_iou(mask_1, mask_2, label):
    """
    Compute the Intersection over Union (IoU) for a given label between two masks.
    """
    # Create binary masks for the given label
    mask_1_label = (mask_1 == label)
    mask_2_label = (mask_2 == label)

    # Compute the intersection and union
    intersection = np.logical_and(mask_1_label, mask_2_label).sum()
    union = np.logical_or(mask_1_label, mask_2_label).sum()

    # Avoid division by zero by checking if union is zero
    if union == 0:
        return 0  # No overlap if there's no region for this label in either image

    # Calculate the IoU (intersection over union)
    iou = intersection / union
    return iou * 100  # Convert to percentage

def compute_overlap_percentages(image_1, image_2, labels):
    """
    Compute the overlap percentages (IoU) for each label in the two images.
    """
    overlap_percentages = {}
    for label in labels:
        iou_percentage = compute_iou(image_1, image_2, label)
        overlap_percentages[label] = iou_percentage
    return overlap_percentages

# Load the NIfTI images
image_1_nii = nib.load('data/segthor_train/train/Patient_27/GT2.nii.gz')  
image_2_nii = nib.load('data/segthor_train/train/Patient_27/GT_corrected.nii.gz')  

# Convert images to NumPy arrays
image_1 = image_1_nii.get_fdata()
image_2 = image_2_nii.get_fdata()

# Labels range from 0 to 4 (background and four segmented organs)
labels = [0, 1, 2, 3, 4]

# Compute the overlap percentages for each label
overlap_percentages = compute_overlap_percentages(image_1, image_2, labels)

# Print the results
for label, iou_percentage in overlap_percentages.items():
    print(f"Label {label}: Overlap (IoU) = {iou_percentage:.2f}%")

Label 0: Overlap (IoU) = 99.99%
Label 1: Overlap (IoU) = 99.85%
Label 2: Overlap (IoU) = 99.37%
Label 3: Overlap (IoU) = 100.00%
Label 4: Overlap (IoU) = 100.00%
